In [1]:
import numpy as np
from torch import nn, optim
import torch
import random
import time

🤔如何加载数据？

In [2]:
with open("labels.txt", "r") as f:
    raw = f.readlines()

tags = []
data = []
for l in raw:
    tags.append(int(l[0]))  # 每行的第一个字符是标签
    d = l[1:-1]  # 去掉标签和换行符
    d = map(float, tuple(d))  # 将字符串转换为tuple，数字转换为float，方便后续转为tensor
    # tuple相对于list更省内存，因为tuple是不可变的，对象所含method更少
    data.append(tuple(d))

# 将标签和数据转为tensor，方便后续切分训练集和测试集
data = torch.tensor(data)
tags = torch.tensor(tags)

# 划分训练集和测试集
train_test_ratio = 0.8
train_size = int(train_test_ratio * len(data))
test_size = len(data) - train_size
data_train = data[:train_size]
data_test = data[train_size:]
tags_train = tags[:train_size]
tags_test = tags[train_size:]

In [3]:
# 直接套用d2l网站上的代码，没有改动
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


batch_size = 10

w = torch.normal(0, 0.01, size=(
    len(data[0]), 1), requires_grad=True, dtype=torch.float32)  # 对每个像素都有一个权重
b = torch.zeros(1, requires_grad=True)

👇方案2：softmax回归

数学原理：

Softmax回归是一种分类算法，它可以将输入数据映射到一个固定维度的输出向量，每个元素对应于输入数据属于各个类别的概率。

- Softmax回归的公式：

$$
\begin{align*}
\hat{y} &= \text{softmax}(Wx+b) \\

\text{softmax}(x_i) &= \frac{\exp(x_i)}{\sum_j \exp(x_j)}
\end{align*}
$$


其中，$W$和$b$是模型的参数，$x$是输入数据，$\hat{y}$是模型的输出。

- Softmax回归的损失函数：

$$
\begin{align*}
L &= -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^K t_{ij}\log\hat{y}_{ij} \\
&= -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^K t_{ij}\log\frac{\exp(x_i^T w_j + b_j)}{\sum_{k=1}^K \exp(x_i^T w_k + b_k)}
\end{align*}
$$

其中，$t_{ij}$是样本$i$的真实标签，$K$是类别数。

- Softmax回归的优化算法：

$$
\begin{align*}
\text{repeat until convergence} \\
\text{for each example} \\
&\text{compute } \hat{y}_i = \text{softmax}(Wx_i+b) \\
&\text{compute } \delta_i = \hat{y}_i - t_i \\
&\text{compute } \nabla_w L = \frac{1}{N}\sum_{i=1}^N \delta_i x_i^T \\
&\text{compute } \nabla_b L = \frac{1}{N}\sum_{i=1}^N \delta_i \\
&\text{update } w = w - \eta \nabla_w L \\
&\text{update } b = b - \eta \nabla_b L
\end{align*}
$$

- [Softmax的实现：](https://zh.d2l.ai/chapter_linear-networks/softmax-regression-concise.html#subsec-softmax-implementation-revisited)

>  尽管我们要计算指数函数，但我们最终在计算交叉熵损失时会取它们的对数。 通过将softmax和交叉熵结合在一
>  起，可以避免反向传播过程中可能会困扰我们的数值稳定性问题。 如下面的等式所示，我们
>避免计算 $\exp(o_j  - \max(o_k))$， 而可以直接使用$o_j - \max(o_k)$，因为$\log(\exp(\cdot))$被抵消了。

>我们也希望保留传统的softmax函数，以备我们需要评估通过模型输出的概率。 但是，我们没有将softmax概率传
>递到损失函数中， 而是在交叉熵损失函数中传递未规范化的预测，并同时计算softmax及其对数， 这是一种类似
>“LogSumExp技巧”的聪明方式。

In [7]:
# 训练

In [6]:
# 测试